In [1]:
#!pip install tensorflow-gpu
#!pip install tensorflow
#!pip install keras

In [13]:
from PIL import Image
import random
import warnings
import os
import shutil
from PIL import ImageFile

from tensorflow import keras

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import StratifiedKFold
keras.__version__

'2.7.0'

In [14]:
base_dir = r"E:\Koding\Python\Kuliah\S2\AI\Projek\Dataset"

training_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validation_dir = os.path.join(base_dir, 'Validation')

In [15]:
sourceFiles = []
classLabels = ['Pneumonia', 'Covid', 'Normal']
picture_size = (224, 224)
batch_size = 32
epoch = 10

In [16]:
def transferBetweenFolders(source, dest, splitRate):
    global sourceFiles
    sourceFiles = os.listdir(source)
    if (len(sourceFiles) != 0):
        transferFileNumbers = int(len(sourceFiles) * splitRate)
        transferIndex = random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source + str(sourceFiles[eachIndex]), dest + str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")


def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(base_dir + '/' + source + '/' + label + '/',
                                base_dir + '/' + dest + '/' + label + '/',
                               splitRate)

In [17]:
# First, check if test folder is empty or not, if not transfer all existing files to train
transferAllClassBetweenFolders('Testing', 'Training', 1.0)
transferAllClassBetweenFolders('Validation', 'Training', 1.0)
# Now, split some part of train data into the test folders.
transferAllClassBetweenFolders('Training', 'Testing', 0.20)

In [18]:
X = []
Y = []

def prepareNameWithLabels(folderName):
    sourceFiles = os.listdir(base_dir + '/Training/' + folderName)
    for val in sourceFiles:
        X.append(val)
        if (folderName == classLabels[0]):
            Y.append(0)
        elif (folderName == classLabels[1]):
            Y.append(1)
        else:
            Y.append(2)
             
# Organize file names and class labels in X and Y variables
for i in range(len(classLabels)):
    prepareNameWithLabels(classLabels[i])

X = np.asarray(X)
Y = np.asarray(Y)

In [19]:
def mobilenetv2():
    base_model = keras.applications.mobilenet_v2.MobileNetV2(include_top=False,
                                                               weights='imagenet',
                                                             input_shape=(224,224,3))

    base_model.trainable = False
    kernel_regulation = keras.regularizers.l2(0.0001)

    x = base_model.output
    # Add some new Fully connected layers to
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Flatten()(x)

    x = keras.layers.Dense(1024, kernel_regularizer=kernel_regulation, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(256, kernel_regularizer=kernel_regulation, activation='relu')(x)
    x = keras.layers.Dense(64, kernel_regularizer=kernel_regulation, activation='relu')(x)

    output_layer = keras.layers.Dense(3, activation='softmax')(x)

    model = keras.models.Model(inputs=base_model.input,outputs=output_layer)

    for layer in model.layers[:90]:
        layer.trainable=False
    for layer in model.layers[90:]:
        layer.trainable=True

    model.summary()

    learning_rate = 0.1
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    losses = 'categorical_crossentropy'

    model.compile(optimizer=optimizer, loss=losses, metrics=['accuracy'])

    return model

In [20]:
cnn = mobilenetv2()

filepath = 'mobilenetv2.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
                                                 verbose=1, save_weights_only=False,
                                                 save_best_only=True, mode='min')

logdir = os.path.join(base_dir, 'logs/')
tfboard = keras.callbacks.TensorBoard(log_dir=logdir)

callbacks_list = [checkpoint, tfboard]

OSError: Unable to open file (truncated file: eof = 40960, sblock->base_addr = 0, stored_eof = 9406464)

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
history_loss = []
history_acc = []
history_val_loss = []
history_val_acc = []
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('Validation', 'Training', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)

    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        if(Y_val[eachIndex]==0):
            classLabel=classLabels[0]
        elif(Y_val[eachIndex]==1):
            classLabel=classLabels[1]
        else:
            classLabel=classLabels[2]
        #Then, copy the validation images to the validation folder
        shutil.move(base_dir+'/Training/'+classLabel+'/'+X_val[eachIndex],
                    base_dir+'/Validation/'+classLabel+'/'+X_val[eachIndex])

    picture_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                           rotation_range=20,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True)
    
    training_set = picture_datagen.flow_from_directory(training_dir,
                                                       shuffle=True,
                                                       target_size=picture_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')

    # Preprocessing the Test set
    test_set = picture_datagen.flow_from_directory(validation_dir,
                                                   shuffle=True,
                                                   target_size=picture_size,
                                                   batch_size=batch_size,
                                                   class_mode='categorical')


    # Training the CNN on the Training set and evaluating it on the Test set
    fit = cnn.fit(training_set,
                    epochs=epoch,
                    validation_data=test_set)
    
    for i in range(epoch):
      history_loss.append(fit.history['loss'][i])
      history_acc.append(fit.history['accuracy'][i])
      history_val_loss.append(fit.history['val_loss'][i])
      history_val_acc.append(fit.history['val_accuracy'][i])

In [ ]:
import pylab as pl
pl.figure()

pl.subplot(121)
pl.plot(history_acc)
pl.ylim([0,1])
pl.title('Accuracy:')
pl.plot(history_val_acc)
pl.legend(('Train', 'Val'))

pl.subplot(122)
pl.plot(history_loss)
pl.ylim([0,1])
pl.title('Loss:')
pl.plot(history_val_loss)
pl.legend(('Train', 'Val'))

In [ ]:
pl.figure()

pl.subplot(121)
pl.ylim([0,1])
pl.plot(history_acc)
pl.title('Accuracy train:')

pl.subplot(122)
pl.ylim([0,1])
pl.title('Accuracy Val:')
pl.plot(history_val_acc)

In [ ]:
pl.figure()

pl.subplot(121)
pl.plot(history_loss)
pl.title('Loss Train:')

pl.subplot(122)
pl.title('Loss Val:')
pl.plot(history_val_loss)

In [ ]:
modelpath = os.path.join(base_dir, filepath)
cnn.save(modelpath)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, confusion_matrix, roc_curve, auc, recall_score
import pylab as pl

def my_metrics(y_true, y_pred, classes, predictions):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average="macro")
    # f1Score=f1_score(y_true, y_pred, average='weighted')
    roc_auc = roc_auc_score(classes, predictions, average='macro', multi_class='ovr')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("Recall : {}".format(recall))
    print("AUC : {}".format(roc_auc))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, roc_auc

In [ ]:
# menghitung akurasi dari training data

picture_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True)
    
training_set = picture_datagen.flow_from_directory(test_dir,
                                                       shuffle=True,
                                                       target_size=picture_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')
score = cnn.evaluate(training_set)

print('Loss: {:0.2f}%'.format(score[0] * 100))
print('Accuracy: {:0.2f}%'.format(score[1] * 100))

In [ ]:
# menghitung akurasi dari training data
predictions = cnn.predict(training_set, verbose=1)

yPredictions = np.argmax(predictions, axis=1)
true_classes = training_set.classes
classes = []
for i in true_classes:
    if(i == 0):
        classes.append([1,0,0])
    elif(i == 1):
        classes.append([0,1,0])
    else:
        classes.append([0,0,1])

my_metrics(true_classes, yPredictions, classes, predictions)